# ETL Project 

---
This project's purpose is to work with a partner to find multiple datasets or data sources and load them together for future analysis. We (Sarah & Paul) are interested in health patterns across the US, so this interest guided our search. Notes are included as we go.


In [15]:
# import dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import pymysql
from sqlalchemy import create_engine
from config import local_sql_dbuser, local_sql_dbpwd 
pymysql.install_as_MySQLdb() 

---
## Import "Welcome" image from web scraping to welcome you to our project

In [44]:
#scraping a web page to bring in an image to welcome you to our project
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

#found the image on google and it linked to this medium article
url = 'https://medium.ip.sx/meet-our-new-project-manager-ovidiu-rauca-263e4d6a6cfc'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [45]:
image = soup.find("img", class_="progressiveMedia-image")["src"]
img_url = image
featured_image_url = img_url

In [47]:
# Use the requests library to download and save the image from the `img_url` above
import requests
import shutil
response = requests.get(img_url, stream=True)
with open('img.jpg', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
    
# Display the image with IPython.display
from IPython.display import Image
Image(url='img.jpg')

---
## Load Mortality Data
We found data on Mortality Rates in the United States from CDC.gov: https://wonder.cdc.gov/controller/datarequest/D140
We decided to export this dataset based on:
* County code
* ICD Chapter (type of death)
* death count
* population of county

The data exported as a txt file. Below I'm reading in the file to pandas like it is a csv, using "\t" to indicate the tab delimited separations between the data cells.

In [7]:
data = pd.read_csv('Deaths.txt', sep="\t")
data.head()

Notes              County  County Code  \
0   NaN  Autauga County, AL       1001.0   
1   NaN  Autauga County, AL       1001.0   
2   NaN  Autauga County, AL       1001.0   
3   NaN  Autauga County, AL       1001.0   
4   NaN  Autauga County, AL       1001.0   

                                         ICD Chapter ICD Chapter Code  Deaths  \
0          Certain infectious and parasitic diseases          A00-B99   211.0   
1                                          Neoplasms          C00-D48  1771.0   
2  Diseases of the blood and blood-forming organs...          D50-D89    44.0   
3      Endocrine, nutritional and metabolic diseases          E00-E88   354.0   
4                   Mental and behavioural disorders          F01-F99   218.0   

   Population Crude Rate % of Total Deaths  
0    918492.0       23.0              0.0%  
1    918492.0      192.8              0.0%  
2    918492.0        4.8              0.0%  
3    918492.0       38.5              0.0%  
4    918492.0       23.7              0.0%

In [8]:
# drop notes column
data.drop(columns=["Notes"], inplace=True)
data.head(20)

County  County Code  \
0   Autauga County, AL       1001.0   
1   Autauga County, AL       1001.0   
2   Autauga County, AL       1001.0   
3   Autauga County, AL       1001.0   
4   Autauga County, AL       1001.0   
5   Autauga County, AL       1001.0   
6   Autauga County, AL       1001.0   
7   Autauga County, AL       1001.0   
8   Autauga County, AL       1001.0   
9   Autauga County, AL       1001.0   
10  Autauga County, AL       1001.0   
11  Autauga County, AL       1001.0   
12  Autauga County, AL       1001.0   
13  Autauga County, AL       1001.0   
14  Autauga County, AL       1001.0   
15  Baldwin County, AL       1003.0   
16  Baldwin County, AL       1003.0   
17  Baldwin County, AL       1003.0   
18  Baldwin County, AL       1003.0   
19  Baldwin County, AL       1003.0   

                                          ICD Chapter ICD Chapter Code  \
0           Certain infectious and parasitic diseases          A00-B99   
1                                           Neoplasms          C00-D48   
2   Diseases of the blood and blood-forming organs...          D50-D89   
3       Endocrine, nutritional and metabolic diseases          E00-E88   
4                    Mental and behavioural disorders          F01-F99   
5                      Diseases of the nervous system          G00-G98   
6                  Diseases of the circulatory system          I00-I99   
7                  Diseases of the respiratory system          J00-J98   
8                    Diseases of the digestive system          K00-K92   
9   Diseases of the musculoskeletal system and con...          M00-M99   
10               Diseases of the genitourinary system          N00-N98   
11  Certain conditions originating in the perinata...          P00-P96   
12  Congenital malformations, deformations and chr...          Q00-Q99   
13  Symptoms, signs and abnormal clinical and labo...          R00-R99   
14         External causes of morbidity and mortality          V01-Y89   
15          Certain infectious and parasitic diseases          A00-B99   
16                                          Neoplasms          C00-D48   
17  Diseases of the blood and blood-forming organs...          D50-D89   
18      Endocrine, nutritional and metabolic diseases          E00-E88   
19                   Mental and behavioural disorders          F01-F99   

    Deaths  Population Crude Rate % of Total Deaths  
0    211.0    918492.0       23.0              0.0%  
1   1771.0    918492.0      192.8              0.0%  
2     44.0    918492.0        4.8              0.0%  
3    354.0    918492.0       38.5              0.0%  
4    218.0    918492.0       23.7              0.0%  
5    366.0    918492.0       39.8              0.0%  
6   2724.0    918492.0      296.6              0.0%  
7    768.0    918492.0       83.6              0.0%  
8    296.0    918492.0       32.2              0.0%  
9     35.0    918492.0        3.8              0.0%  
10   199.0    918492.0       21.7              0.0%  
11    43.0    918492.0        4.7              0.0%  
12    24.0    918492.0        2.6              0.0%  
13   153.0    918492.0       16.7              0.0%  
14   678.0    918492.0       73.8              0.0%  
15   626.0   3102984.0       20.2              0.0%  
16  7323.0   3102984.0      236.0              0.0%  
17   119.0   3102984.0        3.8              0.0%  
18  1021.0   3102984.0       32.9              0.0%  
19  1384.0   3102984.0       44.6              0.0%

## Grouping
Since we're going to need to load this data into a SQL database, we should reduce the amount of records in our current databases to reduce runtime for this specific assignment. It is ok to summarize our current databases since we are not doing any analysis in this assignment.

In [20]:
#group data to reduce the number of records

data_grouped = data.groupby(["County Code"])
data_grouped.head()

County  County Code  \
0         Autauga County, AL       1001.0   
1         Autauga County, AL       1001.0   
2         Autauga County, AL       1001.0   
3         Autauga County, AL       1001.0   
4         Autauga County, AL       1001.0   
15        Baldwin County, AL       1003.0   
16        Baldwin County, AL       1003.0   
17        Baldwin County, AL       1003.0   
18        Baldwin County, AL       1003.0   
19        Baldwin County, AL       1003.0   
31        Barbour County, AL       1005.0   
32        Barbour County, AL       1005.0   
33        Barbour County, AL       1005.0   
34        Barbour County, AL       1005.0   
35        Barbour County, AL       1005.0   
46           Bibb County, AL       1007.0   
47           Bibb County, AL       1007.0   
48           Bibb County, AL       1007.0   
49           Bibb County, AL       1007.0   
50           Bibb County, AL       1007.0   
61         Blount County, AL       1009.0   
62         Blount County, AL       1009.0   
63         Blount County, AL       1009.0   
64         Blount County, AL       1009.0   
65         Blount County, AL       1009.0   
77        Bullock County, AL       1011.0   
78        Bullock County, AL       1011.0   
79        Bullock County, AL       1011.0   
80        Bullock County, AL       1011.0   
81        Bullock County, AL       1011.0   
...                      ...          ...   
44316  Sweetwater County, WY      56037.0   
44317  Sweetwater County, WY      56037.0   
44318  Sweetwater County, WY      56037.0   
44319  Sweetwater County, WY      56037.0   
44320  Sweetwater County, WY      56037.0   
44332       Teton County, WY      56039.0   
44333       Teton County, WY      56039.0   
44334       Teton County, WY      56039.0   
44335       Teton County, WY      56039.0   
44336       Teton County, WY      56039.0   
44344       Uinta County, WY      56041.0   
44345       Uinta County, WY      56041.0   
44346       Uinta County, WY      56041.0   
44347       Uinta County, WY      56041.0   
44348       Uinta County, WY      56041.0   
44358    Washakie County, WY      56043.0   
44359    Washakie County, WY      56043.0   
44360    Washakie County, WY      56043.0   
44361    Washakie County, WY      56043.0   
44362    Washakie County, WY      56043.0   
44369      Weston County, WY      56045.0   
44370      Weston County, WY      56045.0   
44371      Weston County, WY      56045.0   
44372      Weston County, WY      56045.0   
44373      Weston County, WY      56045.0   
44380                    NaN          NaN   
44381                    NaN          NaN   
44382                    NaN          NaN   
44383                    NaN          NaN   
44384                    NaN          NaN   

                                             ICD Chapter ICD Chapter Code  \
0              Certain infectious and parasitic diseases          A00-B99   
1                                              Neoplasms          C00-D48   
2      Diseases of the blood and blood-forming organs...          D50-D89   
3          Endocrine, nutritional and metabolic diseases          E00-E88   
4                       Mental and behavioural disorders          F01-F99   
15             Certain infectious and parasitic diseases          A00-B99   
16                                             Neoplasms          C00-D48   
17     Diseases of the blood and blood-forming organs...          D50-D89   
18         Endocrine, nutritional and metabolic diseases          E00-E88   
19                      Mental and behavioural disorders          F01-F99   
31             Certain infectious and parasitic diseases          A00-B99   
32                                             Neoplasms          C00-D48   
33     Diseases of the blood and blood-forming organs...          D50-D89   
34         Endocrine, nutritional and metabolic diseases          E00-E88   
35                      Mental and behavioural disorders          F01-F99   

## Load FIPS Data

In [10]:
fips_df = pd.read_csv('fipscode_breakout.csv')
fips_df.head()

state  state_fips  county_fips          county class_code
0    AL           1            1  Autauga County         H1
1    AL           1            3  Baldwin County         H1
2    AL           1            5  Barbour County         H1
3    AL           1            7     Bibb County         H1
4    AL           1            9   Blount County         H1

## Load into SQL

insert local connections then read in data to SQL

In [36]:
# Connect to local database'
rds_connection_string = f"{local_sql_dbuser}:{local_sql_dbpwd}@localhost:3306/etl_mortality"
engine = create_engine(f'mysql://{rds_connection_string}')

In [38]:
data.to_sql(name='mortality_data', con=engine, if_exists='append', index=False)

In [39]:
# check for table names
engine.table_names()

['mortality_data']

In [ ]:
# Confirm data has been added by querying the mortality_data table
pd.read_sql_query('select * from mortality_data', con=engine).head()

In [ ]:
data.to_sql(name='mortality_data', con=engine, if_exists='append', index=False)